# Multi variable comparison
Assessing both wave height and wind speed at the same time

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from fmskill import ModelResult, PointObservation, TrackObservation

In [2]:
%load_ext autoreload
%autoreload 2

## Define observations

In [3]:
# wave height
o1 = PointObservation('../tests/testdata/SW/HKNA_Hm0.dfs0', item=0, x=4.2420, y=52.6887, name="HKNA_Hm0")
o2 = PointObservation("../tests/testdata/SW/eur_Hm0.dfs0", item=0, x=3.2760, y=51.9990, name="EPL_Hm0")
o3 = TrackObservation("../tests/testdata/SW/Alti_c2_Dutch.dfs0", item=3, name="c2_Hm0")

# wind speed
wind1 = PointObservation('../tests/testdata/SW/HKNA_wind.dfs0', item=0, x=4.2420, y=52.6887, name="HKNA_wind")
wind2 = PointObservation('../tests/testdata/SW/F16_wind.dfs0', item=0, x=4.01222, y=54.1167, name="F16_wind")
wind3 = TrackObservation("../tests/testdata/SW/Alti_c2_Dutch.dfs0", item=2, name="c2_wind")

In [4]:
o1.variable_name

'Significant_wave_height'

## Define model results

In [5]:
mr1 = ModelResult('../tests/testdata/SW/HKZN_local_2017_DutchCoast.dfsu', name='SW_1')
mr2 = ModelResult('../tests/testdata/SW/HKZN_local_2017_DutchCoast_v2.dfsu', name='SW_2')

In [6]:
mr1.dfs.items

[Sign. Wave Height <Significant wave height> (meter),
 Max. Wave Height <Maximum wave height> (meter),
 Peak Wave Period <Wave period> (second),
 Wave Period, T01 <Wave period> (second),
 Wave Period, T02 <Wave period> (second),
 Peak Wave Direction <Wave direction> (radian),
 Mean Wave Direction <Mean Wave Direction> (degree),
 Dir. Stand. Deviation <Directional Standard Deviation> (degree),
 x-comp. of wave height vector <u velocity component> (meter per sec),
 y-comp. of wave height vector <v velocity component> (meter per sec),
 Surface elevation <Surface Elevation> (meter),
 Current velocity, U <u velocity component> (meter per sec),
 Current velocity, V <v velocity component> (meter per sec),
 Wind speed <Wind speed> (meter per sec),
 Wind direction <Wind Direction> (degree)]

## Associate model and observations and extract

In [7]:
#mr = [mr1, mr2]
mr = mr1

In [8]:
mr.add_observation(o1, item='Sign. Wave Height')
mr.add_observation(o2, item='Sign. Wave Height')
mr.add_observation(wind1, item='Wind speed')
mr.observations

{'HKNA_Hm0': PointObservation: HKNA_Hm0, x=4.242, y=52.6887,
 'EPL_Hm0': PointObservation: EPL_Hm0, x=3.276, y=51.999,
 'HKNA_wind': PointObservation: HKNA_wind, x=4.242, y=52.6887}

In [9]:
cc = mr.extract() 

In [10]:
cc.n_variables

2

## Analysis

In [11]:
cc.skill(variable='Wind_speed')

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
HKNA_wind,277,-0.880928,1.276179,0.923363,1.023092,0.962845,0.064528,0.992046


In [48]:
cc.skill()

,,n,bias,rmse,urmse,mae,cc,si,r2
observation,variable,,,,,,,,
EPL_Hm0,Significant_wave_height,66,-0.075335,0.216357,0.202817,0.183641,0.972467,0.073902,0.993785
HKNA_Hm0,Significant_wave_height,385,-0.195266,0.352283,0.293214,0.251992,0.971082,0.088488,0.988697
HKNA_wind,Wind_speed,277,-0.880928,1.276179,0.923363,1.023092,0.962845,0.064528,0.992046


## mean_skill

In [71]:
cc.mean_skill(variable=0)

,n,bias,rmse,urmse,mae,cc,si,r2
SW_1,451,-0.135301,0.28432,0.248016,0.217816,0.971775,0.081195,0.991241


In [72]:
cc.mean_skill()

,n,bias,rmse,urmse,mae,cc,si,r2
variable,,,,,,,,
Significant_wave_height,451,-0.135301,0.284320,0.248016,0.217816,0.971775,0.081195,0.991241
Wind_speed,277,-0.880928,1.276179,0.923363,1.023092,0.962845,0.064528,0.992046
